In [1]:
library(SCENT)
library(Signac)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(parallel)
library(data.table)
library(dplyr)

Warning message:
“replacing previous import ‘Hmisc::capitalize’ by ‘R.utils::capitalize’ when loading ‘SCENT’”
Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/SCENT/BMMC5.Dendritic.2500.250324/pairs.E2G.res.rds"
path.pairs.E2G.reuse = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/SCENT/BMMC5.Dendritic.25000.240612/pairs.E2G.res.rds"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_BMMC5/Dendritic/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/3.merge_cell_type.240522/RNA_count_matrix/Dendritic.rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/SCENT/BMMC5.Dendritic.25000.250329/"
celltype = "Dendritic"

In [3]:
n.cores = 8

Import candidate E-G pairs

In [4]:
pairs.E2G = readRDS(path.pairs.E2G)

In [5]:
pairs.E2G.reuse = readRDS(path.pairs.E2G.reuse)

Import ATAC matrix

In [6]:
matrix.atac = readRDS(path.matrix.atac_count)
matrix.atac = BinarizeCounts(matrix.atac)

Import RNA matrix

In [7]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

Map gene names

In [8]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [9]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)

Prepare SCENT meta data

In [10]:
meta.data = data.frame(cell = colnames(matrix.rna),
                       nUMI = colSums(matrix.rna),
                       nMito = colSums(matrix.rna[grep("MT-",rownames(matrix.rna)),]),
                       celltype = celltype)
meta.data[,"log.nUMI"] = log(meta.data[,"nUMI"])
meta.data[,"percent.mito"] = meta.data[,"nMito"] / meta.data[,"nUMI"]
meta.data

,cell,nUMI,nMito,celltype,log.nUMI,percent.mito
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
s1d1_CGGGCTTAGTTGGGCC-1,s1d1_CGGGCTTAGTTGGGCC-1,4108,69,Dendritic,8.320692,0.016796495
s1d1_GCGCCTTGTGCATTAG-1,s1d1_GCGCCTTGTGCATTAG-1,4749,99,Dendritic,8.465689,0.020846494
s1d1_AACCGGCTCAACCAAC-1,s1d1_AACCGGCTCAACCAAC-1,3337,42,Dendritic,8.112827,0.012586155
s1d1_GCTCTGTTCTTGTCTG-1,s1d1_GCTCTGTTCTTGTCTG-1,3464,367,Dendritic,8.150179,0.105946882
s1d1_GCGCTTGCAACAGCCT-1,s1d1_GCGCTTGCAACAGCCT-1,7028,249,Dendritic,8.857657,0.035429710
s1d1_CCTAAAGGTTTACCGT-1,s1d1_CCTAAAGGTTTACCGT-1,3765,68,Dendritic,8.233503,0.018061089
s1d1_CAGGCTGGTCTAACAG-1,s1d1_CAGGCTGGTCTAACAG-1,3205,57,Dendritic,8.072467,0.017784711
s1d1_CGCTCCATCGCCTGTT-1,s1d1_CGCTCCATCGCCTGTT-1,10145,243,Dendritic,9.224736,0.023952686
s1d1_CGATTTGCACTGGCTG-1,s1d1_CGATTTGCACTGGCTG-1,3281,38,Dendritic,8.095904,0.011581835


RUN SCENT prediction

In [11]:
RhpcBLASctl::blas_set_num_threads(1)
Sys.setenv(OPENBLAS_NUM_THREADS=1)
SCENT_algorithm.modified = function (object, 
                                     # celltype, 
                                     # ncores, 
                                     # regr = "poisson", 
                                     # bin = TRUE) 
                                     regr = "poisson") {
    res <- data.frame()
    for (n in 1:nrow(object@peak.info)) {        
        gene <- object@peak.info[n, 1]
        this_peak <- object@peak.info[n, 2]
        atac_target <- data.frame(cell = colnames(object@atac), 
            atac = object@atac[this_peak, ])
        # if (bin) {
        #     atac_target[atac_target$atac > 0, ]$atac <- 1
        # }
        mrna_target <- object@rna[gene, ]
        df <- data.frame(cell = names(mrna_target), exprs = as.numeric(mrna_target))
        df <- merge(df, atac_target, by = "cell")
        df <- merge(df, object@meta.data, by = "cell")
        # df2 <- df[df[[object@celltypes]] == celltype, ]
        df2 <- df
        # nonzero_m <- length(df2$exprs[df2$exprs > 0])/length(df2$exprs)
        # nonzero_a <- length(df2$atac[df2$atac > 0])/length(df2$atac)
        # if (nonzero_m > 0.05 & nonzero_a > 0.05) {
            res_var <- "exprs"
            pred_var <- c("atac", object@covariates)
            formula <- as.formula(paste(res_var, paste(pred_var, 
                collapse = "+"), sep = "~"))
            if (regr == "poisson") {
                base = glm(formula, family = "poisson", data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_poisson
                assoc <- SCENT::assoc_poisson
            }
            else if (regr == "negbin") {
                base = glm.nb(formula, data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_negbin
                assoc <- SCENT::assoc_negbin
            }
            # bs = boot::boot(df2, assoc, R = 100, formula = formula, 
            #     stype = "i", parallel = "no", ncpus = 1)
            # p0 = basic_p(bs$t0[1], bs$t[, 1])
            # p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # if (p0 < 0.1) {
            #     bs = boot::boot(df2, assoc, R = 500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.05) {
            #     bs = boot::boot(df2, assoc, R = 2500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.01) {
                bs = boot::boot(df2, assoc, R = 25000, formula = formula, 
                  stype = "i", 
                  # parallel = "multicore", ncpus = ncores)
                  parallel = "no", ncpus = 1)
                # p0 = basic_p(bs$t0[1], bs$t[, 1])
                p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.001) {
            #     bs = boot::boot(df2, assoc, R = 50000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            out <- data.frame(gene = gene, peak = this_peak, 
                beta = coefs[1], se = coefs[2], z = coefs[3], 
                p = coefs[4], boot_basic_p = p0)           
            res <- rbind(res, out)
        # }
    }
    object@SCENT.result <- res
    return(object)
}

In [12]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

In [13]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G)))
chr.run = chr.run[!chr.run %in% chr.done]

Filter genes which can be reused

In [14]:
map_gene_names_reuse <- function(gene_gtf_path, abc_genes_path){
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()
    abc_genes.reuse = abc_genes[abc_genes$abc_name == abc_genes$gene_ref_name,]
	return(abc_genes.reuse)
}

In [15]:
abc_genes.reuse = map_gene_names_reuse(gene_gtf_path, abc_genes_path)

In [16]:
dim(abc_genes.reuse)
abc_genes.reuse

[1] 19669     3

abc_name,Ensembl_ID,gene_ref_name
<chr>,<chr>,<chr>
FAM138A,ENSG00000237613,FAM138A
FAM138F,ENSG00000282591,FAM138F
OR4F5,ENSG00000186092,OR4F5
FAM87B,ENSG00000177757,FAM87B
LINC00115,ENSG00000225880,LINC00115
LINC01128,ENSG00000228794,LINC01128
FAM41C,ENSG00000230368,FAM41C
SAMD11,ENSG00000187634,SAMD11
NOC2L,ENSG00000188976,NOC2L


In [17]:
pairs.E2G.reuse.25000 = pairs.E2G.reuse[pairs.E2G.reuse$boot_basic_p < 0.01 & 
                                        pairs.E2G.reuse$TargetGene %in% abc_genes.reuse$gene_ref_name]

In [18]:
names(pairs.E2G.reuse.25000) = pairs.E2G.reuse.25000$PairName

In [19]:
pairs.E2G.reuse.25000

GRanges object with 23119 ranges and 11 metadata columns:
                                  seqnames              ranges strand |
                                     <Rle>           <IRanges>  <Rle> |
   chr1-100046477-100047244_SASS6     chr1 100046477-100047244      * |
     chr1-10066573-10067824_KIF1B     chr1   10066573-10067824      * |
  chr1-100894613-100896989_CDC14A     chr1 100894613-100896989      * |
    chr1-101238820-101239071_RTCA     chr1 101238820-101239071      * |
  chr1-101247540-101248452_CDC14A     chr1 101247540-101248452      * |
                              ...      ...                 ...    ... .
         chrX-7926909-7928046_STS     chrX     7926909-7928046      * |
       chrX-9352376-9353440_TBL1X     chrX     9352376-9353440      * |
       chrX-9456284-9457059_TBL1X     chrX     9456284-9457059      * |
       chrX-9676955-9677839_TBL1X     chrX     9676955-9677839      * |
       chrX-989889-990634_SLC25A6     chrX       989889-990634      * |
      

In [20]:
table(pairs.E2G[pairs.E2G$boot_basic_p < 0.01]$PairName %in% pairs.E2G.reuse.25000$PairName)


FALSE  TRUE 
 3312 22247 

In [21]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr = pairs.E2G[seqnames(pairs.E2G) == chr.tmp]
  pairs.E2G.chr.res.0 = pairs.E2G.chr[!is.na(pairs.E2G.chr$boot_basic_p) & pairs.E2G.chr$boot_basic_p < 0.01]
  pairs.E2G.chr.keep.0 = pairs.E2G.chr[!pairs.E2G.chr$PairName %in% pairs.E2G.chr.res.0$PairName]
  pairs.E2G.chr.res = pairs.E2G.chr.res.0[!pairs.E2G.chr.res.0$PairName %in% pairs.E2G.reuse.25000$PairName]
  pairs.E2G.chr.keep.1 = pairs.E2G.reuse.25000[pairs.E2G.reuse.25000$PairName %in% pairs.E2G.chr.res.0$PairName]

  gene_peak.chr = as.data.frame(mcols(pairs.E2G.chr.res)[,c("TargetGene","PeakName")])
  list.gene_peak.chr <- split(gene_peak.chr, seq_len(min(n.cores,floor(length(pairs.E2G.chr.res)/2))))
  
  list.obj.SCENT.chr <- lapply(list.gene_peak.chr, function(gene_peak.tmp) {
    obj.SCENT.tmp <- CreateSCENTObj(
      rna = matrix.rna.rename[rownames(matrix.rna.rename) %in% gene_peak.tmp$TargetGene,],
      atac = matrix.atac[rownames(matrix.atac) %in% gene_peak.tmp$PeakName,], 
      meta.data = meta.data,
      peak.info = gene_peak.tmp,
      covariates = c("log.nUMI","percent.mito"), 
      celltypes = "celltype"
    )
  })
  
  cl <- makeCluster(min(n.cores,floor(length(pairs.E2G.chr.res)/2)))
  clusterExport(cl, varlist=c("SCENT_algorithm.modified"))
  list.SCENT.result.chr <- parLapply(cl, list.obj.SCENT.chr, function(obj.SCENT.tmp) {
    obj.SCENT.tmp <- SCENT_algorithm.modified(object = obj.SCENT.tmp)
    return(obj.SCENT.tmp@SCENT.result)
  })
  stopCluster(cl)
  
  
  df.SCENT.result.chr = do.call(rbind,list.SCENT.result.chr)
  rownames(df.SCENT.result.chr) = paste(df.SCENT.result.chr$peak,
                                        df.SCENT.result.chr$gene,
                                        sep = "_")

  pairs.E2G.chr.res = pairs.E2G.chr.res[pairs.E2G.chr.res$PairName %in% rownames(df.SCENT.result.chr)]
  mcols(pairs.E2G.chr.res)[,c("beta","se","z","p","boot_basic_p")] = 
    df.SCENT.result.chr[pairs.E2G.chr.res$PairName,c("beta","se","z","p","boot_basic_p")]
  pairs.E2G.chr.res$boot_basic_p.log10 = -log10(pairs.E2G.chr.res$boot_basic_p)
  pairs.E2G.chr.res$sign = 1
  pairs.E2G.chr.res$sign[pairs.E2G.chr.res$beta < 0] = -1
  pairs.E2G.chr.res$boot_basic_p.log10.signed = pairs.E2G.chr.res$boot_basic_p.log10 * pairs.E2G.chr.res$sign
  pairs.E2G.chr.merge = c(pairs.E2G.chr.res,
                          pairs.E2G.chr.keep.0,
                          pairs.E2G.chr.keep.1)
  saveRDS(pairs.E2G.chr.merge,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.merge)
}

[1] "chr1"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr10"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr11"


...

Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr9"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chrX"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


Merge results

In [22]:
chr.merge = as.character(unique(seqnames(pairs.E2G)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [23]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 533980 ranges and 11 metadata columns:
                                  seqnames              ranges strand |
                                     <Rle>           <IRanges>  <Rle> |
                   chr1.chr1.chr1     chr1     1001820-1002126      * |
                   chr1.chr1.chr1     chr1 100712622-100713915      * |
                   chr1.chr1.chr1     chr1 101025254-101026528      * |
                   chr1.chr1.chr1     chr1 108557323-108557589      * |
                   chr1.chr1.chr1     chr1 108956405-108957155      * |
                              ...      ...                 ...    ... .
    chrX.chrX-7926909-7928046_STS     chrX     7926909-7928046      * |
  chrX.chrX-9352376-9353440_TBL1X     chrX     9352376-9353440      * |
  chrX.chrX-9456284-9457059_TBL1X     chrX     9456284-9457059      * |
  chrX.chrX-9676955-9677839_TBL1X     chrX     9676955-9677839      * |
  chrX.chrX-989889-990634_SLC25A6     chrX       989889-990634      * |
     

In [24]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = celltype
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta","se","z","p","boot_basic_p",
                         "boot_basic_p.log10",
                         "sign",
                         "boot_basic_p.log10.signed")]
df.output[,"FDR"] = p.adjust(df.output$boot_basic_p, method = "BH")
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,se,z,p,boot_basic_p,boot_basic_p.log10,sign,boot_basic_p.log10.signed,FDR
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,1001820,1002126,CDK11B,Dendritic,0.3448225,0.13342005,2.584488,0.0097523722,0.01088,1.963371,1,1.963371,0.2257332
chr1,100712622,100713915,AGL,Dendritic,0.4888488,0.17838169,2.740465,0.0061352273,0.01128,1.947691,1,1.947691,0.2270713
chr1,101025254,101026528,CDC14A,Dendritic,0.1573107,0.04306560,3.652816,0.0002593801,0.01024,1.989700,1,1.989700,0.2200845
chr1,108557323,108557589,TAF13,Dendritic,0.5417223,0.18988329,2.852922,0.0043319200,0.01112,1.953895,1,1.953895,0.2257332
chr1,108956405,108957155,FAM102B,Dendritic,-0.4808983,0.13269581,-3.624065,0.0002900082,0.00912,2.040005,-1,-2.040005,0.2085073
chr1,109397321,109398342,SARS,Dendritic,-0.6303272,0.18274499,-3.449217,0.0005622137,0.00392,2.406714,-1,-2.406714,0.1295058
chr1,109493723,109494615,TAF13,Dendritic,0.3443703,0.12613283,2.730219,0.0063292197,0.00952,2.021363,1,2.021363,0.2144118
chr1,109509030,109510127,STXBP3,Dendritic,0.2988347,0.11037132,2.707539,0.0067784067,0.01224,1.912219,1,1.912219,0.2349865
chr1,110726488,110727426,ATP5F1,Dendritic,-1.3517732,0.35652418,-3.791533,0.0001497202,0.01320,1.879426,-1,-1.879426,0.2424265


In [25]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] dplyr_1.1.4          data.table_1.15.2    Matrix_1.6-5        
 [4] GenomicRanges_1.54.1 GenomeInfoDb_1.38.1  IRanges